In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def filter_words(story):
    story = story.split(' ')
    for i in story:
        if(i in stop_words):
            story.remove(i)
    return story

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df_key = pd.read_csv('/Users/karth/OneDrive/Desktop/OELP/test/keywords.csv',delimiter='\t')
df  = pd.read_csv('/Users/karth/OneDrive/Desktop/OELP/test/features.csv',delimiter='\t')

s = set()                           # set of all unique keywords
for i in df_key['k_h']:
    x = eval(i)
    for j in x.keys():
        s.add(j)
#print(s)                               
M = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df_key['k_h']:
    x = eval(i)
    l = []
    for j in s:
        if(j in x.keys()):
            l.append(x[j])
        else:
            l.append(0)
    M.append(l)
mat = np.array(M)
sim_mat = cosine_similarity(mat)          # first similarity matrix which takes only keywords and its helpful score as features
#print(sim_mat)

In [3]:

df['avg_rating'] = df['avg_rating'].astype(str)
features = ['director','cast','avg_rating','writer','title']    # these are the features for second similarity matrix

def combine(df_row):                            # combining all the unique features in single line
    x = ''
    for i in features[:]:
        x+=df_row[i]
        x+=','
    story_arr = filter_words(df_row['storyline'])
    for i in story_arr:
        x+=i
        x+=','
    return x

cv = CountVectorizer()                                     # this counts frequency of each unique feature in each movie
df['final'] = df.apply(combine,axis=1)
count_matrix = cv.fit_transform(df['final'])
cos_sim = cosine_similarity(count_matrix)
#print(cv.get_feature_names_out())
#print(count_matrix.toarray())
#print(cos_sim)


In [5]:
final = []                                                 # This is the final matrix which is combination of both the above matrices
for i in range(len(sim_mat)):
    row = []                                               # both rows and columns represents corresponding movies and the value in that entry will be final similarity score of those 2 movies
    for j in range(len(sim_mat[0])):
        b = ((0.6*sim_mat[i][j])+(0.4*cos_sim[i][j]))/2    # we are considering more probability (0.6) to first matrix because giving more priority to keywords and their helpful score will give more accurate recommendation
        row.append(b)                      # final similarity score = [(0.6)*(similarity score from keywords alone) + (0.4)*(similarity score from remaining features)] 
    final.append(row)                 # values 0.6 and 0.4 were decided by multiple trails and these values gave more accurate recommendation

titles = list(df['title'])                      # now from index of final matrix we will get titlee and story
story = list(df['storyline'])
t_i = {}
i_t = {}
for i in range(len(titles)):
    t_i[titles[i]] = i
    i_t[i] = titles[i]

def find(word):
    ind = t_i[word]
    inds = []
    m = list(final[ind]).copy()
    for i in range(11):
        x = max(m)                             # we are recommending top 10 highest similarity scores
        print(x)
        j = m.index(x)
        print(m.index(x))
        m[j] = -2
        inds.append(j)
    for i in inds[1:]:
        print(i_t[i],'  ',story[i])
find('Interstellar')


0.5000000000000003
1604
0.09366172991700071
498
0.08792304317977813
1298
0.08502263415391632
902
0.08425540135365252
1473
0.08393251018367007
243
0.08104650259457144
1555
0.07719373413458974
280
0.0743324842964186
35
0.07018936628063739
1399
0.06973305149813867
1477
First Man    A look at the life of the astronaut, Neil Armstrong, and the legendary space mission that led him to become the first man to walk on the Moon on July 20, 1969.
Why Him?    A holiday gathering threatens to go off the rails when Ned Fleming realizes that his daughter's Silicon Valley millionaire boyfriend is about to pop the question.
Steve Jobs    Steve Jobs takes us behind the scenes of the digital revolution, to paint a portrait of the man at its epicenter. The story unfolds backstage at three iconic product launches, ending in 1998 with the unveiling of the iMac.
On the Rocks    A young mother reconnects with her larger-than-life playboy father on an adventure through New York.
My Big Fat Greek Wedding 2    A